In [1]:
import pandas as pd
import sqlite3

In [2]:
base_iptu = pd.read_excel('dados/valores_sc.xlsx', )

In [3]:
base_iptu.columns.values[0] = 'setor'  # Renomeia a primeira coluna

In [4]:
base_iptu.head()

,setor,unidades,valor_terreno_medio,valor_construcao_medio,area_construida_media,area_terreno_media
0,355030801000001P,220,1954.426230,1921.486339,171.262295,188.732240
1,355030801000002P,838,1693.710145,1908.429952,369.101449,217.405797
2,355030801000003P,175,1794.658228,1922.240506,181.170886,205.917722
3,355030801000004P,130,1676.136364,1927.920455,198.147727,260.306818
4,355030801000005P,184,1929.859259,1912.992593,208.451852,260.622222


In [5]:
def carregar_dados_sqlite(arquivo_db, tabela):
    """
    Lê dados de uma tabela em um banco de dados SQLite e os transforma em um DataFrame do pandas.

    Parâmetros:
        arquivo_db (str): Caminho para o arquivo SQLite.
        tabela (str): Nome da tabela no banco de dados.

    Retorna:
        pd.DataFrame: DataFrame contendo os dados da tabela.
    """
    try:
        # Conexão com o banco de dados
        conexao = sqlite3.connect(arquivo_db)
        # Query para buscar todos os dados
        query = f"SELECT * FROM {tabela}"
        # Leitura dos dados em um DataFrame
        df = pd.read_sql_query(query, conexao)
        # Fechando a conexão
        conexao.close()
        return df
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None


In [6]:
arquivo_db = "dados_final.db"
tabela = "dados_final"
df_scraping = carregar_dados_sqlite(arquivo_db, tabela)


In [7]:
df_scraping.head()

,setor,preco_medio_m2,quantidade_imoveis,maior_preco_m2,menor_preco_m2,lat,lon
0,355030801000005P,4814.814815,1,4814.814815,4814.814815,-23.571521,-46.570585
1,355030801000074P,5175.438596,1,5175.438596,5175.438596,-23.577912,-46.572691
2,355030801000104P,9948.305448,6,12000.000000,8928.571429,-23.556107,-46.580224
3,355030801000131P,5672.713530,2,6530.612245,4814.814815,-23.570261,-46.571982
4,355030801000197P,9499.664333,14,11996.000000,8108.108108,-23.557752,-46.579734


In [8]:
def unificar_tabelas(df_sql, df_xlsx, coluna_setor):
    """
    Combina dois DataFrames pelo setor censitário e realiza um agrupamento.

    Parâmetros:
        df_sql (pd.DataFrame): DataFrame extraído do banco de dados SQL.
        df_xlsx (pd.DataFrame): DataFrame carregado do arquivo Excel.
        coluna_setor (str): Nome da coluna que representa o setor censitário.

    Retorna:
        pd.DataFrame: DataFrame unificado com informações agrupadas.
    """
    # Unificar as tabelas pelo setor censitário
    df_merged = pd.merge(df_sql, df_xlsx, on=coluna_setor, how="inner")
    
    # Realizar um agrupamento por setor, se necessário
    df_grouped = df_merged.groupby(coluna_setor).agg("mean").reset_index()
    
    return df_grouped

# Exemplo de uso
# Supondo que df_sql e df_xlsx sejam os DataFrames:
# df_unificado = unificar_tabelas(df_sql, df_xlsx, "setor_censitario")


In [9]:
df_unificado = unificar_tabelas(df_scraping, base_iptu, "setor")

In [10]:
df_unificado.head()

,setor,preco_medio_m2,quantidade_imoveis,maior_preco_m2,menor_preco_m2,lat,lon,unidades,valor_terreno_medio,valor_construcao_medio,area_construida_media,area_terreno_media
0,355030801000005P,4814.814815,1.0,4814.814815,4814.814815,-23.571521,-46.570585,184.0,1929.859259,1912.992593,208.451852,260.622222
1,355030801000074P,5175.438596,1.0,5175.438596,5175.438596,-23.577912,-46.572691,179.0,1707.290698,1954.627907,159.348837,216.244186
2,355030801000104P,9948.305448,6.0,12000.000000,8928.571429,-23.556107,-46.580224,266.0,2262.575758,1945.575758,239.903030,195.509091
3,355030801000131P,5672.713530,2.0,6530.612245,4814.814815,-23.570261,-46.571982,359.0,1829.363128,1908.234637,256.513966,207.720670
4,355030801000197P,9499.664333,14.0,11996.000000,8108.108108,-23.557752,-46.579734,132.0,2416.809524,1937.104762,169.076190,145.676190


In [11]:
df_unificado.shape

(1258, 12)

In [23]:
df_unificado.rename(columns={'preco_medio_m2': 'preco_medio_m2_qa', 'maior_preco_m2':'maior_preco_qa',
                             'menor_preco_m2': 'menor_preco_qa', 'valor_terreno_medio':'preco_medio_m2_terreno_iptu',
                             'valor_construcao_medio':'preco_medio_m2_construcao_iptu',
                             'setor':'CD_SETOR'}, inplace=True)

In [24]:
df_unificado.head()

,CD_SETOR,preco_medio_m2_qa,quantidade_imoveis,maior_preco_qa,menor_preco_qa,lat,lon,unidades,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,area_construida_media,area_terreno_media
0,355030801000005P,4814.814815,1.0,4814.814815,4814.814815,-23.571521,-46.570585,184.0,1929.859259,1912.992593,208.451852,260.622222
1,355030801000074P,5175.438596,1.0,5175.438596,5175.438596,-23.577912,-46.572691,179.0,1707.290698,1954.627907,159.348837,216.244186
2,355030801000104P,9948.305448,6.0,12000.000000,8928.571429,-23.556107,-46.580224,266.0,2262.575758,1945.575758,239.903030,195.509091
3,355030801000131P,5672.713530,2.0,6530.612245,4814.814815,-23.570261,-46.571982,359.0,1829.363128,1908.234637,256.513966,207.720670
4,355030801000197P,9499.664333,14.0,11996.000000,8108.108108,-23.557752,-46.579734,132.0,2416.809524,1937.104762,169.076190,145.676190


In [26]:
df_teste = df_unificado[['CD_SETOR', 'preco_medio_m2_qa', 'preco_medio_m2_terreno_iptu', 'preco_medio_m2_construcao_iptu', 'lat', 'lon']]

In [27]:
df_teste.head()

,CD_SETOR,preco_medio_m2_qa,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,lat,lon
0,355030801000005P,4814.814815,1929.859259,1912.992593,-23.571521,-46.570585
1,355030801000074P,5175.438596,1707.290698,1954.627907,-23.577912,-46.572691
2,355030801000104P,9948.305448,2262.575758,1945.575758,-23.556107,-46.580224
3,355030801000131P,5672.713530,1829.363128,1908.234637,-23.570261,-46.571982
4,355030801000197P,9499.664333,2416.809524,1937.104762,-23.557752,-46.579734


In [28]:
df_teste['diferenca_qa_iptu_terreno'] = df_teste['preco_medio_m2_qa'] - df_teste['preco_medio_m2_terreno_iptu']

C:\Users\ggsna\AppData\Local\Temp\ipykernel_11412\203411054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste['diferenca_qa_iptu_terreno'] = df_teste['preco_medio_m2_qa'] - df_teste['preco_medio_m2_terreno_iptu']


In [29]:
df_teste.head()

,CD_SETOR,preco_medio_m2_qa,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,lat,lon,diferenca_qa_iptu_terreno
0,355030801000005P,4814.814815,1929.859259,1912.992593,-23.571521,-46.570585,2884.955556
1,355030801000074P,5175.438596,1707.290698,1954.627907,-23.577912,-46.572691,3468.147899
2,355030801000104P,9948.305448,2262.575758,1945.575758,-23.556107,-46.580224,7685.729691
3,355030801000131P,5672.713530,1829.363128,1908.234637,-23.570261,-46.571982,3843.350401
4,355030801000197P,9499.664333,2416.809524,1937.104762,-23.557752,-46.579734,7082.854809


In [30]:
df_teste['diff_percentual'] = ( df_teste['preco_medio_m2_qa'] / df_teste['diferenca_qa_iptu_terreno']) * 100

C:\Users\ggsna\AppData\Local\Temp\ipykernel_11412\1655758279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste['diff_percentual'] = ( df_teste['preco_medio_m2_qa'] / df_teste['diferenca_qa_iptu_terreno']) * 100


In [31]:
df_teste.head()

,CD_SETOR,preco_medio_m2_qa,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,lat,lon,diferenca_qa_iptu_terreno,diff_percentual
0,355030801000005P,4814.814815,1929.859259,1912.992593,-23.571521,-46.570585,2884.955556,166.893899
1,355030801000074P,5175.438596,1707.290698,1954.627907,-23.577912,-46.572691,3468.147899,149.227736
2,355030801000104P,9948.305448,2262.575758,1945.575758,-23.556107,-46.580224,7685.729691,129.438659
3,355030801000131P,5672.713530,1829.363128,1908.234637,-23.570261,-46.571982,3843.350401,147.598135
4,355030801000197P,9499.664333,2416.809524,1937.104762,-23.557752,-46.579734,7082.854809,134.121969


In [32]:
import folium 
from folium.plugins import HeatMap

df_heatmap_qa = df_teste[['lat', 'lon', 'preco_medio_m2_qa']]
max_value = df_heatmap_qa['preco_medio_m2_qa'].max()
df_heatmap_qa = df_heatmap_qa.dropna()

m = folium.Map(location=[-23.5505, -46.6333], zoom_start=11)
heat_data = [[row['lat'],row['lon'], row['preco_medio_m2_qa']/max_value] for index, row in df_heatmap_qa.iterrows()]
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

m.save('heatmap_qa.html')

In [33]:
df_heatmap_iptu = df_teste[['lat', 'lon', 'preco_medio_m2_terreno_iptu']]
df_heatmap_iptu = df_heatmap_iptu.dropna()

m = folium.Map(location=[-23.5505, -46.6333], zoom_start=11)
heat_data = [[row['lat'],row['lon'], row['preco_medio_m2_terreno_iptu']] for index, row in df_heatmap_iptu.iterrows()]
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

m.save('heatmap_iptu.html')

In [34]:
import geopandas as gpd

In [36]:
setores = gpd.read_file("dados\geo\SP_Malha_Preliminar_2022.shp")
sp = setores[setores["NM_MUN"] == "São Paulo"]
sp = sp.to_crs(epsg=4326)
sp.head()

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\ggsna\AppData\Local\Temp\ipykernel_11412\532264347.py:1: SyntaxWarning: invalid escape sequence '\g'
  setores = gpd.read_file("dados\geo\SP_Malha_Preliminar_2022.shp")


,CD_SETOR,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,NM_MICRO,CD_MESO,NM_MESO,CD_RGI,NM_RGI,CD_RGINT,NM_RGINT,CD_CONCURB,NM_CONCURB,geometry
65763,355030801000001P,0.071797,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,"POLYGON ((-46.56954 -23.56918, -46.57016 -23.5..."
65764,355030801000002P,0.071902,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,"POLYGON ((-46.56806 -23.56521, -46.56786 -23.5..."
65765,355030801000003P,0.055681,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,"POLYGON ((-46.56619 -23.56605, -46.56632 -23.5..."
65766,355030801000004P,0.064905,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,"POLYGON ((-46.56876 -23.56856, -46.56863 -23.5..."
65767,355030801000005P,0.086820,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,...,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,"POLYGON ((-46.57121 -23.57005, -46.57056 -23.5..."


In [37]:
print(df_unificado.columns)
print(sp.columns)

Index(['CD_SETOR', 'preco_medio_m2_qa', 'quantidade_imoveis', 'maior_preco_qa',
       'menor_preco_qa', 'lat', 'lon', 'unidades',
       'preco_medio_m2_terreno_iptu', 'preco_medio_m2_construcao_iptu',
       'area_construida_media', 'area_terreno_media'],
      dtype='object')
Index(['CD_SETOR', 'AREA_KM2', 'CD_REGIAO', 'NM_REGIAO', 'CD_UF', 'NM_UF',
       'CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST',
       'CD_MICRO', 'NM_MICRO', 'CD_MESO', 'NM_MESO', 'CD_RGI', 'NM_RGI',
       'CD_RGINT', 'NM_RGINT', 'CD_CONCURB', 'NM_CONCURB', 'geometry'],
      dtype='object')


In [39]:
merged = df_unificado.merge(sp[['CD_SETOR', 'geometry']], on='CD_SETOR', how='left')
geo_merged = gpd.GeoDataFrame(merged, geometry='geometry')
geo_merged.set_crs(sp.crs, inplace=True)
geo_merged.head()


,CD_SETOR,preco_medio_m2_qa,quantidade_imoveis,maior_preco_qa,menor_preco_qa,lat,lon,unidades,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,area_construida_media,area_terreno_media,geometry
0,355030801000005P,4814.814815,1.0,4814.814815,4814.814815,-23.571521,-46.570585,184.0,1929.859259,1912.992593,208.451852,260.622222,"POLYGON ((-46.57121 -23.57005, -46.57056 -23.5..."
1,355030801000074P,5175.438596,1.0,5175.438596,5175.438596,-23.577912,-46.572691,179.0,1707.290698,1954.627907,159.348837,216.244186,"POLYGON ((-46.57146 -23.57796, -46.57163 -23.5..."
2,355030801000104P,9948.305448,6.0,12000.000000,8928.571429,-23.556107,-46.580224,266.0,2262.575758,1945.575758,239.903030,195.509091,"POLYGON ((-46.57809 -23.55652, -46.57821 -23.5..."
3,355030801000131P,5672.713530,2.0,6530.612245,4814.814815,-23.570261,-46.571982,359.0,1829.363128,1908.234637,256.513966,207.720670,"POLYGON ((-46.56954 -23.56918, -46.56952 -23.5..."
4,355030801000197P,9499.664333,14.0,11996.000000,8108.108108,-23.557752,-46.579734,132.0,2416.809524,1937.104762,169.076190,145.676190,"POLYGON ((-46.57821 -23.55725, -46.57807 -23.5..."


In [55]:
# Criar uma função para categorizar os preços
def categorizar_preco(preco):
    if preco <= 2000:
        return "0-2000"
    elif preco <= 4000:
        return "2001-4000"
    elif preco <= 6000:
        return "4001-6000"
    elif preco <= 8000:
        return "6001-8000"
    elif preco <= 10000:
        return "8001-10000"
    elif preco <= 12000:
        return "10001-12000"
    elif preco <= 14000:
        return "12001-14000"
    elif preco <= 16000:
        return "14001-16000"
    elif preco <= 18000:
        return "16001-18000"
    elif preco <= 20000:
        return "18001-20000"
    else:
        return "20001+"

# Aplicar a categorização à coluna `preco_m2`
geo_merged['faixa_preco'] = geo_merged['preco_medio_m2_qa'].apply(categorizar_preco)

# Criar um dicionário de cores para as categorias
cor_categorias = {
    "0-2000": "#f7fbff",       # Azul claro
    "2001-4000": "#deebf7",
    "4001-6000": "#c6dbef",
    "6001-8000": "#9ecae1",
    "8001-10000": "#6baed6",
    "10001-12000": "#4292c6",
    "12001-14000": "#2171b5",
    "14001-16000": "#084594",  # Azul escuro
    "16001-18000": "#08306b",  # Azul ainda mais escuro
    "18001-20000": "#041f50",  # Azul quase preto
    "20001+": "#000033"        # Preto com tonalidade azulada
}

# Criar o mapa base
m = folium.Map(location=[-23.5505, -46.6333], zoom_start=11)

# Adicionar os setores ao mapa com cores personalizadas
for _, row in geo_merged.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, row=row: {
            'fillColor': cor_categorias[row['faixa_preco']],
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7
        },
        tooltip=f"Setor: {row['CD_SETOR']}<br>Preço/m²: R${row['preco_medio_m2_qa']:.2f}<br>Faixa: {row['faixa_preco']}"
    ).add_to(m)

# Salvar o mapa como arquivo HTML
m.save("mapa_coropletico_qa.html")

print("Mapa coroplético salvo como 'mapa_coropletico_qa.html'. Abra no navegador para visualizar.")


Mapa coroplético salvo como 'mapa_coropletico_qa.html'. Abra no navegador para visualizar.


In [93]:
# Aplicar a categorização à coluna `preco_m2`
geo_merged['faixa_preco'] = geo_merged['preco_medio_m2_terreno_iptu'].apply(categorizar_preco)

# Criar o mapa base
m = folium.Map(location=[-23.5505, -46.6333], zoom_start=11)

# Adicionar os setores ao mapa com cores personalizadas
for _, row in geo_merged.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, row=row: {
            'fillColor': cor_categorias[row['faixa_preco']],
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7
        },
        tooltip=f"Setor: {row['CD_SETOR']}<br>Preço/m²: R${row['preco_medio_m2_terreno_iptu']:.2f}<br>Faixa: {row['faixa_preco']}"
    ).add_to(m)

# Salvar o mapa como arquivo HTML
m.save("mapa_coropletico_iptu.html")

print("Mapa coroplético salvo como 'mapa_coropletico_iptu.html'. Abra no navegador para visualizar.")


Mapa coroplético salvo como 'mapa_coropletico_iptu.html'. Abra no navegador para visualizar.


# Testando ITBI

In [86]:
lotes = gpd.read_file("dados/combinado/lotes_combinados.shp")
lotes.head()


,lo_setor,lo_quadra,lo_lote,lo_condomi,lo_tp_quad,lo_tp_lote,geometry
0,052,053,0015,00,F,F,"POLYGON ((338796.77596 7393110.6844, 338802.60..."
1,052,078,0070,00,F,F,"POLYGON ((339772.27003 7392871.97321, 339772.2..."
2,053,045,0038,00,F,F,"POLYGON ((340427.17206 7392801.40436, 340429.5..."
3,052,024,0147,00,F,F,"POLYGON ((338698.99645 7392112.56282, 338691.2..."
4,052,024,0148,00,F,F,"POLYGON ((338698.99645 7392112.56282, 338695.4..."


In [87]:
print(lotes.crs)


LOCAL_CS["Undefined Cartesian SRS with unknown unit",UNIT["unknown",0],AXIS["X",OTHER],AXIS["Y",OTHER]]


In [88]:
# Definir o CRS original, substitua "EPSG:XXXX" pelo CRS correto
lotes.set_crs("EPSG:31983", inplace=True, allow_override=True)


,lo_setor,lo_quadra,lo_lote,lo_condomi,lo_tp_quad,lo_tp_lote,geometry
0,052,053,0015,00,F,F,"POLYGON ((338796.776 7393110.684, 338802.606 7..."
1,052,078,0070,00,F,F,"POLYGON ((339772.27 7392871.973, 339772.204 73..."
2,053,045,0038,00,F,F,"POLYGON ((340427.172 7392801.404, 340429.553 7..."
3,052,024,0147,00,F,F,"POLYGON ((338698.996 7392112.563, 338691.275 7..."
4,052,024,0148,00,F,F,"POLYGON ((338698.996 7392112.563, 338695.432 7..."
...,...,...,...,...,...,...,...
1677975,115,037,0048,00,F,F,"POLYGON ((356461.627 7395469.613, 356456.724 7..."
1677976,193,046,0007,00,F,F,"POLYGON ((357832.528 7396011.824, 357838.681 7..."
1677977,193,046,0015,00,F,F,"POLYGON ((357852.413 7395995.586, 357856.661 7..."
1677978,193,014,0018,00,F,F,"POLYGON ((357723.091 7396368.457, 357722.316 7..."


In [89]:
lotes_teste = lotes.to_crs(epsg=4326)


In [90]:
lotes_teste.head()

,lo_setor,lo_quadra,lo_lote,lo_condomi,lo_tp_quad,lo_tp_lote,geometry
0,052,053,0015,00,F,F,"POLYGON ((-46.5795 -23.5644, -46.57945 -23.564..."
1,052,078,0070,00,F,F,"POLYGON ((-46.56997 -23.56665, -46.56998 -23.5..."
2,053,045,0038,00,F,F,"POLYGON ((-46.56357 -23.56735, -46.56354 -23.5..."
3,052,024,0147,00,F,F,"POLYGON ((-46.58057 -23.5734, -46.58064 -23.57..."
4,052,024,0148,00,F,F,"POLYGON ((-46.58057 -23.5734, -46.58061 -23.57..."


In [91]:
lotes["SQL"] = lotes["lo_setor"].str.zfill(3) + \
                      lotes["lo_quadra"].str.zfill(3) + \
                      lotes["lo_lote"].str.zfill(4)

print(lotes[["lo_setor", "lo_quadra", "lo_lote", "SQL"]].head())


  lo_setor lo_quadra lo_lote         SQL
0      052       053    0015  0520530015
1      052       078    0070  0520780070
2      053       045    0038  0530450038
3      052       024    0147  0520240147
4      052       024    0148  0520240148


In [92]:
lotes_filtrado = lotes[['SQL', 'geometry']]

In [75]:
itbi = pd.read_excel("dados/itbi_2024.xlsx", sheet_name="JAN-2024")

In [76]:
itbi.head()

,SQL,Nome do Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza de Transação,Valor de Transação (declarado pelo contribuinte),Data de Transação,...,Situação do SQL,Área do Terreno (m2),Testada (m),Fração Ideal,Área Construída (m2),Uso (IPTU),Descrição do uso (IPTU),Padrão (IPTU),Descrição do padrão (IPTU),ACC (IPTU)
0,8424500301,R GALIA,190,NaN,NaN,NaN,5602000,4.Arrematação (em leilão ou hasta pública),30459304.00,2023-10-26,...,Ativo Predial,8182,310.55,1.0000,7888,10,RESIDÊNCIA,15,RESIDENCIAL HORIZONTAL,2004.0
1,4212800055,R LOEFGREN,1401,UN 1404,VILA CLEMENTINO,NaN,4040031,2.Cessão de direitos relativos a compromisso d...,127130.82,2023-12-19,...,Ativo Predial,263,7.50,1.0000,385,31,"PRÉDIO DE ESCRITÓRIO OU CONSULTÓRIO, NÃO EM CO...",32,COMERCIAL HORIZONTAL,1962.0
2,1401200516,R AUGUSTA,2898,E 2900 UN 216,NaN,NaN,1412100,2.Cessão de direitos relativos a compromisso d...,181891.33,2023-12-19,...,Ativo Predial,278,5.05,1.0000,540,40,LOJA,33,COMERCIAL HORIZONTAL,1994.0
3,9035514653,AV WASHINGTON LUIS,1576,AP 152 E 4VGS,JD MARAJOARA,ED JEQUITIBA,4662902,8.Cessão de direitos hereditários (ou sucessór...,316666.66,2023-12-21,...,Ativo Predial,73836,89.91,0.0022,615,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),25,RESIDENCIAL VERTICAL,1992.0
4,10302548768,AV GUIDO CALOI,99999,"APT 211, TORRE A",SANTO AMARO,NaN,5802140,1.Compra e venda,299118.26,2023-12-20,...,Ativo Territorial,31351,227.54,1.0000,0,0,TERRENO,0,TERRENO,NaN
